# HW 6 - Tim Demetriades
## Latent Factor Matrix Factorization
4/3/2021

### Part 1 - 5,000 Steps

In [1]:
import numpy as np                  # for arrays 

Below is the Results array `r` which holds ratings on a scale from 1 to 5 for 4 movies by 5 differen users. Not all users have ratings for all movies. 

In [2]:
r = np.array([[5, 3, 0, 1],
              [4, 0, 0, 1],
              [1, 1, 0, 5],
              [1, 0, 0, 4],
              [0, 1, 5, 4]])
print('Results Matrix:')
r

Results Matrix:


array([[5, 3, 0, 1],
       [4, 0, 0, 1],
       [1, 1, 0, 5],
       [1, 0, 0, 4],
       [0, 1, 5, 4]])

In [3]:
num_row_r = r.shape[0]
num_col_r = r.shape[1]
print(f'r has {num_row_r} rows and {num_col_r} columns.')

r has 5 rows and 4 columns.


In [4]:
k = 2    # Number of latent factors/features (hyperparameter)
# Fill p and q with random floats between 0 and 1
p = np.random.rand(num_row_r, k)
q = np.random.rand(num_col_r, k)    # easier to leave in this shape and transpose later
print('P Matrix:')
print(p)
print('Q Matrix:')
print(q.T)    # q is transposed

P Matrix:
[[0.96499347 0.71909942]
 [0.57431202 0.54379492]
 [0.6196881  0.89047171]
 [0.1819025  0.55888606]
 [0.03872846 0.94662023]]
Q Matrix:
[[0.39889836 0.17875725 0.87286966 0.49876861]
 [0.25642355 0.92588058 0.61750797 0.44081818]]


In [5]:
# Hyper parameters
steps = 5000     # epochs
alpha = 0.0002   # learning rate
beta = 0.02      # for regularization to avoid over fitting and prevent extreme values

Below is the main loop where the stochastic gradient descent occurs.

In [6]:
def matrix_factorization(r, p, q, k, steps, alpha, beta):
    # List of training samples (non-zero values from ratings matrix)
    # i,j are the coordinates of r
    # r[i,j] is the rating at the corresonding coordinates
    samples = [
        (i, j, r[i,j])
        for i in range(num_row_r)
        for j in range(num_col_r)
        if r[i,j] > 0
    ]

    # Main gradient descent loop (repeats for number of steps)
    for i in range(steps):

        # Stochastic Gradient Descent 
        for i, j, rating in samples:    # for every rating with a non-zero value

            # Get prediction and error
            prediction = np.matmul(p[i, :], q[j, :])
            e = (rating - prediction)

            # Update p and q matrices
            p[i, :] += alpha * (e * q[j, :] - beta * p[i, :])    # update p for all columns in current row where row is current row in r
            q[j, :] += alpha * (e * p[i, :] - beta * q[j, :])    # update q for all columns in current row where row is current column in r
            
    return p, q.T    # q is transposed

Here we call the function to get our new P and Q matrices.

In [7]:
[new_p, new_q] = matrix_factorization(r, p, q, k, steps, alpha, beta)

In [8]:
print('New P Matrix:')
print(new_p)
print('New Q Matrix:')
print(new_q)

New P Matrix:
[[2.22882524 0.59114137]
 [1.69526024 0.53317542]
 [0.12545045 2.10258715]
 [0.15683517 1.69752564]
 [0.8199803  1.79523908]]
New Q Matrix:
[[ 2.17286678  1.11533115  1.42989857 -0.15866866]
 [ 0.37593773  0.30475212  2.02693002  2.33837723]]


Now that we have the new P and Q matrices, we simply multiply them together to get the predicted ratings matrix (which includes predictions for the zero values in the actual ratings matrix).  

In [9]:
r_predicted = np.matmul(new_p, new_q)
print('Predicted Ratings')
print(r_predicted.round(4))
print('Actual Ratings')
print(r)

Predicted Ratings
[[5.0652 2.666  4.3852 1.0287]
 [3.884  2.0533 3.5048 0.9778]
 [1.063  0.7807 4.4412 4.8967]
 [0.9789 0.6922 3.665  3.9446]
 [2.4566 1.4617 4.8113 4.0678]]
Actual Ratings
[[5 3 0 1]
 [4 0 0 1]
 [1 1 0 5]
 [1 0 0 4]
 [0 1 5 4]]


Above we can see the predicted ratings matrix and the actual ratings matrix. Notice that the non-zero values in the actual ratings matrix match the corresponding values in the predicted ratings matrix. This means the function worked pretty well. For zero values in the actual ratings matrix, the predicted ratings matrix now includes values, with these values corresponding to the predicted ratings for the movies in which a user did not have a rating for in the actual ratings matrix. 

Let's try this with 10,000 steps now.

### Part 2 - 10,000 Steps

Let's see what happens if we do 5,000 more steps with the same matrices for a grand total of 10,000 steps. Ideally this should improve our predicted ratings matrix.

In [10]:
[new_p, new_q] = matrix_factorization(r, new_p, new_q.T, k, steps, alpha, beta)    # transposing new_q

In [11]:
print('New P Matrix:')
print(new_p)
print('New Q Matrix:')
print(new_q)

New P Matrix:
[[2.26960418 0.58580665]
 [1.78865631 0.55334928]
 [0.20293499 2.17613728]
 [0.21535667 1.74847039]
 [0.60210853 1.80036563]]
New Q Matrix:
[[ 2.11553231  1.21650951  1.49510781 -0.14732335]
 [ 0.29244213  0.26824799  2.22310152  2.28128457]]


In [12]:
r_predicted = np.matmul(new_p, new_q)
print('Predicted Ratings')
print(r_predicted.round(4))
print('Actual Ratings')
print(r)

Predicted Ratings
[[4.9727 2.9181 4.6956 1.002 ]
 [3.9458 2.3244 3.9044 0.9988]
 [1.0657 0.8306 5.1412 4.9345]
 [0.9669 0.731  4.209  3.957 ]
 [1.8003 1.2154 4.9026 4.0184]]
Actual Ratings
[[5 3 0 1]
 [4 0 0 1]
 [1 1 0 5]
 [1 0 0 4]
 [0 1 5 4]]


The non-zero values in the actual ratings matrix appear to match even closer to the corresponding values in the predicted ratings matrix. By adding 5,000 steps we have improved the accuracy of the predictions. 